In [1]:
import tensorflow as tf
import numpy as np


In [ ]:
tf.reset_default_graph()

sentence = [
    'Lorem ipsum dolor sit amet consectetur adipisicing elit',
    'sed do eiusmod tempor incididunt ut labore et dolore magna',
    'aliqua Ut enim ad minim veniam quis nostrud exercitation'
    ]

word_dict = ' '.join(sentence).split()
word_dict = list(set(word_dict))
word_num = {w:i for i,w in enumerate(word_dict)}
num_word = {i:w for i,w in enumerate(word_dict)}
n_class = len(word_dict)
n_hidden = 5
n_step = len(word_dict)

def get_batch(sentence):
    input_batch = []
    target_batch = []
    words = ' '.join(sentence).split()
    for i,w in enumerate(words[:-1]):
        input_temp = [word_num[w] for w in words[:(i+1)]]
        input_temp = input_temp + [0]*(n_step-len(input_temp))
        target_temp = word_num[words[i+1]]
        
        input_batch.append(np.eye(n_class)[input_temp])
        target_batch.append(np.eye(n_class)[target_temp])
    return input_batch,target_batch

X = tf.placeholder(tf.float32,[None,n_step,n_class])
Y = tf.placeholder(tf.float32,[None,n_class])

#model
fw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)
bw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

W = tf.Variable(tf.random_normal([n_hidden*2,n_class]))
b = tf.Variable(tf.random_normal([n_class]))

outputs,_ = tf.nn.bidirectional_dynamic_rnn(fw_cell,bw_cell,X,dtype=tf.float32)

output = tf.concat((outputs[0],outputs[1]),2)
output = tf.transpose(output,[1,0,2])
output = output[-1]

h = tf.matmul(output,W)+b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=h))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

prediction = tf.cast(tf.argmax(h, 1), tf.int32)

input_batch,target_batch = get_batch(sentence)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(5000):
        _,loss = sess.run([optimizer,cost],feed_dict={X:input_batch,Y:target_batch})
        if (i+1) % 1000 == 0:
            print('epoch:%d cost:%.6f' % ((i+1),loss))
        
    predict =  sess.run([prediction], feed_dict={X: input_batch})
    print(predict[0])
    print([num_word[n] for n in [pre for pre in predict[0]]])

epoch:1000 cost:1.276349
epoch:2000 cost:0.706782
epoch:3000 cost:0.436345
epoch:4000 cost:0.269067
